In [3]:
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

data = [['', 'Emma', 'Isabella', 'Ava', 'Olivia', 'Sophia', 'row-sum'],
        ['Emma', 16, 3, 28, 0, 18, 65],
        ['Isabella', 18, 0, 12, 5, 29, 64],
        ['Ava', 9, 11, 17, 27, 0, 64],
        ['Olivia', 19, 0, 31, 11, 12, 73],
        ['Sophia', 23, 17, 10, 0, 34, 84]]

table = ff.create_table(data, index=True)
# py.iplot(table, filename='Data-Table')

In [4]:
import numpy as np

matrix=np.array([[16,  3, 28,  0, 18],
                 [18,  0, 12,  5, 29],
                 [ 9, 11, 17, 27,  0],
                 [19,  0, 31, 11, 12],
                 [23, 17, 10,  0, 34]], dtype=int)

def check_data(data_matrix):
    L, M=data_matrix.shape
    if L!=M:
        raise ValueError('Data array must have (n,n) shape')
    return L

L=check_data(matrix)

In [5]:
PI=np.pi

def moduloAB(x, a, b): #maps a real number onto the unit circle identified with 
                       #the interval [a,b), b-a=2*PI
        if a>=b:
            raise ValueError('Incorrect interval ends')
        y=(x-a)%(b-a)
        return y+b if y<0 else y+a

def test_2PI(x):
    return 0<= x <2*PI

In [6]:
row_sum=[np.sum(matrix[k,:]) for k in range(L)]

#set the gap between two consecutive ideograms
gap=2*PI*0.005
ideogram_length=2*PI*np.asarray(row_sum)/sum(row_sum)-gap*np.ones(L)

In [7]:
def get_ideogram_ends(ideogram_len, gap):
    ideo_ends=[]
    left=0
    for k in range(len(ideogram_len)):
        right=left+ideogram_len[k]
        ideo_ends.append([left, right])
        left=right+gap
    return ideo_ends

ideo_ends=get_ideogram_ends(ideogram_length, gap)
ideo_ends

[[0, 1.1354613447974538],
 [1.1668772713333517, 2.284386658110292],
 [2.31580258464619, 3.43331197142313],
 [3.464727897959028, 4.7438049069205865],
 [4.775220833456484, 6.251769380643687]]

In [8]:
def make_ideogram_arc(R, phi, a=50):
    # R is the circle radius
    # phi is the list of ends angle coordinates of an arc
    # a is a parameter that controls the number of points to be evaluated on an arc
    if not test_2PI(phi[0]) or not test_2PI(phi[1]):
        phi=[moduloAB(t, 0, 2*PI) for t in phi]
    length=(phi[1]-phi[0])% 2*PI
    nr=5 if length<=PI/4 else int(a*length/PI)

    if phi[0] < phi[1]:
        theta=np.linspace(phi[0], phi[1], nr)
    else:
        phi=[moduloAB(t, -PI, PI) for t in phi]
        theta=np.linspace(phi[0], phi[1], nr)
    return R*np.exp(1j*theta)

In [10]:
z=make_ideogram_arc(1.3, [11*PI/6, PI/17])
print(z)

[1.12583302-0.65j       1.14814501-0.60972373j 1.16901672-0.5686826j
 1.18842197-0.5269281j  1.20633642-0.48451259j 1.22273759-0.44148929j
 1.23760491-0.39791217j 1.25091973-0.3538359j  1.26266534-0.30931575j
 1.27282702-0.26440759j 1.28139202-0.21916775j 1.28834958-0.17365297j
 1.29369099-0.12792036j 1.29740954-0.08202728j 1.29950058-0.0360313j
 1.29996146+0.01000988j 1.29879163+0.0560385j  1.29599253+0.10199682j
 1.2915677 +0.1478272j  1.28552267+0.19347214j 1.27786503+0.23887437j]


In [11]:
labels=['Emma', 'Isabella', 'Ava', 'Olivia', 'Sophia']
ideo_colors=['rgba(244, 109, 67, 0.75)',
             'rgba(253, 174, 97, 0.75)',
             'rgba(254, 224, 139, 0.75)',
             'rgba(217, 239, 139, 0.75)',
             'rgba(166, 217, 106, 0.75)']#brewer colors with alpha set on 0.75

In [12]:
def map_data(data_matrix, row_value, ideogram_length):
    mapped=np.zeros(data_matrix.shape)
    for j  in range(L):
        mapped[:, j]=ideogram_length*data_matrix[:,j]/row_value
    return mapped

mapped_data=map_data(matrix, row_sum, ideogram_length)
mapped_data

array([[0.27949818, 0.05240591, 0.48912181, 0.        , 0.31443545],
       [0.31429952, 0.        , 0.20953301, 0.08730542, 0.50637144],
       [0.15714976, 0.19207193, 0.29683843, 0.47144927, 0.        ],
       [0.33291045, 0.        , 0.54316969, 0.19273763, 0.21025923],
       [0.40429305, 0.2988253 , 0.17577959, 0.        , 0.5976506 ]])

In [13]:
idx_sort=np.argsort(mapped_data, axis=1)
idx_sort

array([[3, 1, 0, 4, 2],
       [1, 3, 2, 0, 4],
       [4, 0, 1, 2, 3],
       [1, 3, 4, 0, 2],
       [3, 2, 1, 0, 4]])

In [15]:
def make_ribbon_ends(mapped_data, ideo_ends,  idx_sort):
    L=mapped_data.shape[0]
    ribbon_boundary=np.zeros((L,L+1))
    for k in range(L):
        start=ideo_ends[k][0]
        ribbon_boundary[k][0]=start
        for j in range(1,L+1):
            J=idx_sort[k][j-1]
            ribbon_boundary[k][j]=start+mapped_data[k][J]
            start=ribbon_boundary[k][j]
    return [[(ribbon_boundary[k][j],ribbon_boundary[k][j+1] ) for j in range(L)] for k in range(L)]

ribbon_ends=make_ribbon_ends(mapped_data, ideo_ends,  idx_sort)
print('ribbon ends starting from the ideogram[2]\n', ribbon_ends[2])


ribbon ends starting from the ideogram[2]
 [(2.31580258464619, 2.31580258464619), (2.31580258464619, 2.472952342161697), (2.472952342161697, 2.6650242680139837), (2.6650242680139837, 2.9618626988766086), (2.9618626988766086, 3.43331197142313)]


In [16]:
def control_pts(angle, radius):
    #angle is a  3-list containing angular coordinates of the control points b0, b1, b2
    #radius is the distance from b1 to the  origin O(0,0) 

    if len(angle)!=3:
        raise InvalidInputError('angle must have len =3')
    b_cplx=np.array([np.exp(1j*angle[k]) for k in range(3)])
    b_cplx[1]=radius*b_cplx[1]
    return zip(b_cplx.real, b_cplx.imag)

In [17]:
def ctrl_rib_chords(l, r, radius):
    # this function returns a 2-list containing control poligons of the two quadratic Bezier
    #curves that are opposite sides in a ribbon
    #l (r) the list of angular variables of the ribbon arc ends defining 
    #the ribbon starting (ending) arc 
    # radius is a common parameter for both control polygons
    if len(l)!=2 or len(r)!=2:
        raise ValueError('the arc ends must be elements in a list of len 2')
    return [control_pts([l[j], (l[j]+r[j])/2, r[j]], radius) for j in range(2)]

In [18]:
ribbon_color=[L*[ideo_colors[k]] for k in range(L)]

In [19]:
ribbon_color[0][4]=ideo_colors[4]
ribbon_color[1][2]=ideo_colors[2]
ribbon_color[2][3]=ideo_colors[3]
ribbon_color[2][4]=ideo_colors[4]

In [20]:
def make_q_bezier(b):# defines the Plotly SVG path for a quadratic Bezier curve defined by the 
                     #list of its control points
    if len(b)!=3:
        raise valueError('control poligon must have 3 points')
    A, B, C=b
    return 'M '+str(A[0])+',' +str(A[1])+' '+'Q '+\
                str(B[0])+', '+str(B[1])+ ' '+\
                str(C[0])+', '+str(C[1])

b=[(1,4), (-0.5, 2.35), (3.745, 1.47)]

make_q_bezier(b)

'M 1,4 Q -0.5, 2.35 3.745, 1.47'

In [21]:
def make_ribbon_arc(theta0, theta1):

    if test_2PI(theta0) and test_2PI(theta1):
        if theta0 < theta1:
            theta0= moduloAB(theta0, -PI, PI)
            theta1= moduloAB(theta1, -PI, PI)
            if theta0*theta1>0:
                raise ValueError('incorrect angle coordinates for ribbon')

        nr=int(40*(theta0-theta1)/PI)
        if nr<=2: nr=3
        theta=np.linspace(theta0, theta1, nr)
        pts=np.exp(1j*theta)# points on arc in polar complex form

        string_arc=''
        for k in range(len(theta)):
            string_arc+='L '+str(pts.real[k])+', '+str(pts.imag[k])+' '
        return   string_arc
    else:
        raise ValueError('the angle coordinates for an arc side of a ribbon must be in [0, 2*pi]')

make_ribbon_arc(np.pi/3, np.pi/6)

'L 0.5000000000000001, 0.8660254037844386 L 0.5877852522924732, 0.8090169943749473 L 0.6691306063588583, 0.7431448254773941 L 0.7431448254773942, 0.6691306063588581 L 0.8090169943749475, 0.5877852522924731 L 0.8660254037844387, 0.49999999999999994 '

In [22]:
def make_layout(title, plot_size):
    axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

    return go.Layout(title=title,
                  xaxis=dict(axis),
                  yaxis=dict(axis),
                  showlegend=False,
                  width=plot_size,
                  height=plot_size,
                  margin=dict(t=25, b=25, l=25, r=25),
                  hovermode='closest',
                  shapes=[]# to this list one appends below the dicts defining the ribbon,
                           #respectively the ideogram shapes
                 )

In [23]:
def make_ideo_shape(path, line_color, fill_color):
    #line_color is the color of the shape boundary
    #fill_collor is the color assigned to an ideogram
    return  dict(
                  line=dict(
                  color=line_color,
                  width=0.45
                 ),

            path=  path,
            type='path',
            fillcolor=fill_color,
            layer='below'
        )


In [24]:
def make_ribbon(l, r, line_color, fill_color, radius=0.2):
    #l=[l[0], l[1]], r=[r[0], r[1]]  represent the opposite arcs in the ribbon 
    #line_color is the color of the shape boundary
    #fill_color is the fill color for the ribbon shape
    poligon=ctrl_rib_chords(l,r, radius)
    b,c =poligon

    return  dict(
                line=dict(
                color=line_color, width=0.5
            ),
            path=  make_q_bezier(b)+make_ribbon_arc(r[0], r[1])+
                   make_q_bezier(c[::-1])+make_ribbon_arc(l[1], l[0]),
            type='path',
            fillcolor=fill_color,
            layer='below'
        )

def make_self_rel(l, line_color, fill_color, radius):
    #radius is the radius of Bezier control point b_1
    b=control_pts([l[0], (l[0]+l[1])/2, l[1]], radius)
    return  dict(
                line=dict(
                color=line_color, width=0.5
            ),
            path=  make_q_bezier(b)+make_ribbon_arc(l[1], l[0]),
            type='path',
            fillcolor=fill_color,
            layer='below'
        )

def invPerm(perm):
    # function that returns the inverse of a permutation, perm
    inv = [0] * len(perm)
    for i, s in enumerate(perm):
        inv[s] = i
    return inv

layout=make_layout('Chord diagram', 400)

In [25]:
radii_sribb=[0.4, 0.30, 0.35, 0.39, 0.12]# these value are set after a few trials 


In [28]:
ribbon_info=[]
for k in range(L):

    sigma=idx_sort[k]
    sigma_inv=invPerm(sigma)
    for j in range(k, L):
        if matrix[k][j]==0 and matrix[j][k]==0: continue
        eta=idx_sort[j]
        eta_inv=invPerm(eta)
        l=ribbon_ends[k][sigma_inv[j]]

        if j==k:
#             layout = list(layout)
            layout['shapes'].append(make_self_rel(l, 'rgb(175,175,175)' ,
                                    ideo_colors[k], radius=radii_sribb[k]))
            z=0.9*np.exp(1j*(l[0]+l[1])/2)
            #the text below will be displayed when hovering the mouse over the ribbon
            text=labels[k]+' commented on '+ '{:d}'.format(matrix[k][k])+' of '+ 'herself Fb posts',
            ribbon_info.append(go.Scatter(x=[z.real],
                                       y=[z.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color=ideo_colors[k]),
                                       text=text,
                                       hoverinfo='text'
                                       )
                              )
        else:
            r=ribbon_ends[j][eta_inv[k]]
            zi=0.9*np.exp(1j*(l[0]+l[1])/2)
            zf=0.9*np.exp(1j*(r[0]+r[1])/2)
            #texti and textf are the strings that will be displayed when hovering the mouse 
            #over the two ribbon ends
            texti=labels[k]+' commented on '+ '{:d}'.format(matrix[k][j])+' of '+\
                  labels[j]+ ' Fb posts',

            textf=labels[j]+' commented on '+ '{:d}'.format(matrix[j][k])+' of '+\
            labels[k]+ ' Fb posts',
            ribbon_info.append(go.Scatter(x=[zi.real],
                                       y=[zi.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color=ribbon_color[k][j]),
                                       text=texti,
                                       hoverinfo='text'
                                       )
                              ),
            ribbon_info.append(go.Scatter(x=[zf.real],
                                       y=[zf.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color=ribbon_color[k][j]),
                                       text=textf,
                                       hoverinfo='text'
                                       )
                              )
            r=(r[1], r[0])#IMPORTANT!!!  Reverse these arc ends because otherwise you get
                          # a twisted ribbon
            #append the ribbon shape
            layout['shapes'].append(make_ribbon(l, r, 'rgb(175,175,175)' , ribbon_color[k][j]))




TypeError: list indices must be integers or slices, not str

In [29]:
ideograms=[]
for k in range(len(ideo_ends)):
    z= make_ideogram_arc(1.1, ideo_ends[k])
    zi=make_ideogram_arc(1.0, ideo_ends[k])
    m=len(z)
    n=len(zi)
    ideograms.append(go.Scatter(x=z.real,
                             y=z.imag,
                             mode='lines',
                             line=dict(color=ideo_colors[k], shape='spline', width=0.25),
                             text=labels[k]+'<br>'+'{:d}'.format(row_sum[k]),
                             hoverinfo='text'
                             )
                     )


    path='M '
    for s in range(m):
        path+=str(z.real[s])+', '+str(z.imag[s])+' L '

    Zi=np.array(zi.tolist()[::-1])

    for s in range(m):
        path+=str(Zi.real[s])+', '+str(Zi.imag[s])+' L '
    path+=str(z.real[0])+' ,'+str(z.imag[0])

    layout['shapes'].append(make_ideo_shape(path,'rgb(150,150,150)' , ideo_colors[k]))

data = go.Data(ideograms+ribbon_info)
fig = go.Figure(data=data, layout=layout)

import plotly.offline as off
off.init_notebook_mode()

off.iplot(fig, filename='chord-diagram-Fb')

TypeError: list indices must be integers or slices, not str

In [30]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

data = go.Data(ribbon_info+ideograms)
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='chord-diagram-Fb')

/Users/anayahall/anaconda/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




ValueError: 
    Invalid value of type 'builtins.list' received for the 'layout' property of 
        Received value: ['angularaxis', 'annotations', 'autosize', 'bargap', 'bargroupgap', 'barmode', 'barnorm', 'boxgap', 'boxgroupgap', 'boxmode', 'calendar', 'colorway', 'datarevision', 'direction', 'dragmode', 'font', 'geo', 'grid', 'height', 'hiddenlabels', 'hiddenlabelssrc', 'hidesources', 'hoverdistance', 'hoverlabel', 'hovermode', 'images', 'legend', 'mapbox', 'margin', 'orientation', 'paper_bgcolor', 'plot_bgcolor', 'polar', 'radialaxis', 'scene', 'selectdirection', 'separators', 'shapes', 'showlegend', 'sliders', 'spikedistance', 'template', 'ternary', 'title', 'titlefont', 'updatemenus', 'violingap', 'violingroupgap', 'violinmode', 'width', 'xaxis', 'yaxis']

    The 'layout' property is an instance of Layout
    that may be specified as:
      - An instance of plotly.graph_objs.Layout
      - A dict of string/value properties that will be passed
        to the Layout constructor



In [2]:
import numpy as np
import plotly.graph_objs as go
# import colorlover as cl


def get_spaced_colors(n, randomized=False):
    if n > 0:
        max_value = 255
        interval = max_value / n
        hues = np.arange(0, max_value, interval)
#         return cl.to_rgb(["hsl(%d,80%%,40%%)" % i for i in hues])
        return None
    else:
        return None


PI = np.pi


def check_square(M):
    d, n = M.shape
    if d != n:
        raise ValueError("Data array must be square.")
    return n


def moduloAB(x, a, b):
    if a >= b:
        raise ValueError('Incorrect inverval ends')
    y = (x - a) % (b - a)
    return y + b if y < 0 else y + a


def test_2PI(x):
    return 0 <= x < 2 * PI


def get_ideogram_ends(ideaogram_len, gap):
    ideo_ends = []
    left = 0
    for k in range(len(ideaogram_len)):
        right = left + ideaogram_len[k]
        ideo_ends.append([left, right])
        left = right + gap
    return ideo_ends


def make_ideogram_arc(R, phi, a=50):
    # R is the circle radius
    # Phi is a list of the ends angle coordinates of an arc
    # a is a parameter that controls the number of points to be evaluated
    if not test_2PI(phi[0]) or not test_2PI(phi[1]):
        phi = [moduloAB(t, 0, 2*PI) for t in phi]
    length = (phi[1] - phi[0]) % 2 * PI
    nr = 5 if length <= PI/4 else int(a * length / PI)
    if phi[0] < phi[1]:
        theta = np.linspace(phi[0], phi[1], nr)
    else:
        phi = [moduloAB(t, -PI, PI) for t in phi]
        theta = np.linspace(phi[0], phi[1], nr)
    return R * np.exp(1j*theta)


def map_data(data_matrix, row_value, ideogram_length):
    n = data_matrix.shape[0]  # square, so same as 1
    mapped = np.zeros([n, n])
    for j in range(n):
        mapped[:, j] = ideogram_length * data_matrix[:, j] / row_value
    return mapped


def make_ribbon_ends(mapped_data, ideo_ends, idx_sort):
    n = mapped_data.shape[0]
    ribbon_boundary = np.zeros((n, n+1))
    for k in range(n):
        start = ideo_ends[k][0]
        ribbon_boundary[k][0] = start
        for j in range(1, n+1):
            J = idx_sort[k][j-1]
            ribbon_boundary[k][j] = start + mapped_data[k][J]
            start = ribbon_boundary[k][j]
    return [[(ribbon_boundary[k][j], ribbon_boundary[k][j+1])
             for j in range(n)] for k in range(n)]


def control_pts(angle, radius):
    if len(angle) != 3:
        raise ValueError('Angle must have len = 3')
    b_cplx = np.array([np.exp(1j*angle[k]) for k in range(3)])
    b_cplx[1] = radius * b_cplx[1]
    return list(zip(b_cplx.real, b_cplx.imag))


def ctrl_rib_chords(l, r, radius):
    if len(l) != 2 or len(r) != 2:
        raise ValueError('The arc ends must be elements in a list of len 2')
    return [control_pts([l[j], (l[j]+r[j])/2, r[j]], radius) for j in range(2)]


def make_q_bezier(b):
    if len(b) != 3:
        raise ValueError('Contaol polygon must have 3 points')
    A, B, C = b
    return 'M ' + str(A[0]) + "," + str(A[1]) + " " + "Q " + \
           str(B[0]) + ", " + str(B[1]) + " " + \
           str(C[0]) + ", " + str(C[1])


def make_ribbon_arc(theta0, theta1):
    if test_2PI(theta0) and test_2PI(theta1):
        if theta0 < theta1:
            theta0 = moduloAB(theta0, -PI, PI)
            theta1 = moduloAB(theta1, -PI, PI)
            if theta0 * theta1 > 0:
                raise ValueError('Incorrect angle coordinates for ribbon')
        nr = int(40 * (theta0 - theta1) / PI)
        if nr <= 2:
            nr = 3
        theta = np.linspace(theta0, theta1, nr)
        pts = np.exp(1j * theta)
        string_arc = ''
        for k in range(len(theta)):
            string_arc += "L " + str(pts.real[k]) + ", " + str(pts.imag[k])+' '
        return string_arc
    else:
        raise ValueError('The angle coords for arc ribbon must be [0, 2*PI]')


def make_layout(title):
    xaxis = dict(showline=False,
                 zeroline=False,
                 showgrid=False,
                 showticklabels=False,
                 title='')
    yaxis = {**xaxis, 'scaleanchor': 'x'}
    return dict(title=title,
                xaxis=xaxis,
                yaxis=yaxis,
                showlegend=False,
                margin=dict(t=25, b=25, l=25, r=25),
                hovermode='closest',
                shapes=[])


def make_ideo_shape(path, line_color, fill_color):
    return dict(
        line=go.Line(color=line_color, width=0.45),
        path=path,
        type='path',
        fillcolor=fill_color,
        layer='below'
    )


def make_ribbon(l, r, line_color, fill_color, radius=0.2):
    poligon = ctrl_rib_chords(l, r, radius)
    b, c = poligon
    return dict(line=go.Line(color=line_color, width=0.5),
                path=make_q_bezier(b) + make_ribbon_arc(r[0], r[1]) +
                make_q_bezier(c[::-1]) + make_ribbon_arc(l[1], l[0]),
                type='path',
                fillcolor=fill_color,
                layer='below')


def make_self_rel(l, line_color, fill_color, radius):
    b = control_pts([l[0], (l[0]+l[1])/2, l[1]], radius)
    return dict(
        line=dict(color=line_color, width=0.5),
        path=make_q_bezier(b) + make_ribbon_arc(l[1], l[0]),
        type='path',
        fillcolor=fill_color,
        layer='below'
    )


def invPerm(perm):
    inv = [0] * len(perm)
    for i, s in enumerate(perm):
        inv[s] = i
    return inv


def make_filled_chord(M):
    n = M.shape[0]
    labels = M.columns
    M = M.T
    matrix = M.as_matrix()
    row_sum = [np.sum(matrix[k, :]) for k in range(n)]
    gap = 2 * PI * 10e-8
    ideogram_length = 2*PI*np.asarray(row_sum)/sum(row_sum) - gap*np.ones(n)
    ideo_colors = [x[:3] + "a" + x[3:-1] + ",.75" + x[-1] for x in
                   get_spaced_colors(len(labels))]
    mapped_data = map_data(M.as_matrix(), row_sum, ideogram_length)
    idx_sort = np.argsort(mapped_data, axis=1)
    ideo_ends = get_ideogram_ends(ideogram_length, gap)
    ribbon_ends = make_ribbon_ends(mapped_data, ideo_ends, idx_sort)
    ribbon_color = [n * [ideo_colors[k]] for k in range(n)]
    layout = make_layout(' ')
    ribbon_info = []
    radii_sribb = [0.2] * n
    for k in range(n):
        sigma = idx_sort[k]
        sigma_inv = invPerm(sigma)
        for j in range(k, n):
            if M.iloc[k, j] == 0 and M.iloc[j, k] == 0:
                continue
            eta = idx_sort[j]
            eta_inv = invPerm(eta)
            l = ribbon_ends[k][sigma_inv[j]]
            if j == k:
                layout['shapes'].append(
                    make_self_rel(l,
                                  'rgb(175,175,175)',
                                  ideo_colors[k],
                                  radius=radii_sribb[k]))
                z = 0.9 * np.exp(1j * (l[0] + l[1]) / 2)
                text = labels[k] + " co-occurs with " + \
                    "{:d}".format(M.iloc[k, k]) + " of its own appearences"
                ribbon_info.append(
                    go.Scatter(x=[z.real],
                               y=[z.imag],
                               mode='markers',
                               text=text,
                               hoverinfo="text",
                               marker=dict(size=0.5,
                                           color=ideo_colors[k])))
            else:
                r = ribbon_ends[j][eta_inv[k]]
                zi = 0.9 * np.exp(1j * (l[0] + l[1]) / 2)
                zf = 0.9 * np.exp(1j * (r[0] + r[1]) / 2)
                texti = labels[k] + " co-occurs with " + \
                    "{:d}".format(matrix[k][j]) + " of the " + \
                    labels[j] + " appearences"
                textf = labels[j] + " co-occurs with " + \
                    "{:d}".format(matrix[j][k]) + " of the " + \
                    labels[k] + " appearences"
                ribbon_info.append(
                    go.Scatter(x=[zi.real],
                               y=[zi.imag],
                               mode='markers',
                               text=texti,
                               hoverinfo="text",
                               marker=dict(size=0.5,
                                           color=ribbon_color[k][j])))
                ribbon_info.append(
                    go.Scatter(x=[zf.real],
                               y=[zf.imag],
                               mode='markers',
                               text=textf,
                               hoverinfo="text",
                               marker=dict(size=0.5,
                                           color=ribbon_color[j][k])))
                r = (r[1], r[0])
                if matrix[k][j] > matrix[j][k]:
                    color_of_highest = ribbon_color[k][j]
                else:
                    color_of_highest = ribbon_color[j][k]
                layout['shapes'].append(
                    make_ribbon(l, r, 'rgb(175, 175, 175)',
                                color_of_highest))
    ideograms = []
    for k in range(len(ideo_ends)):
        z = make_ideogram_arc(1.1, ideo_ends[k])
        zi = make_ideogram_arc(1.0, ideo_ends[k])
        m = len(z)
        n = len(zi)
        ideograms.append(
            go.Scatter(x=z.real,
                       y=z.imag,
                       mode='lines',
                       line=dict(color=ideo_colors[k],
                                 shape='spline',
                                 width=0.25),
                       text=labels[k]+'<br>'+'{:d}'.format(row_sum[k]),
                       hoverinfo='text'))
        path = 'M '
        for s in range(m):
            path += str(z.real[s]) + ', ' + str(z.imag[s]) + ' L '
        Zi = np.array(zi.tolist()[::-1])
        for s in range(m):
            path += str(Zi.real[s]) + ", " + str(Zi.imag[s]) + ' L '
        path += str(z.real[0]) + ' ,' + str(z.imag[0])
        layout['shapes'].append(make_ideo_shape(path,
                                                'rgb(150,150,150)',
                                                ideo_colors[k]))
    data = ideograms + ribbon_info
    fig = {
        "data": data,
        "layout": layout
    }
    return fig

In [3]:
make_filled_chord(matrix)

NameError: name 'matrix' is not defined